# Instruções para o uso do Notebook
Este notebook documenta uma série de testes conduzidos para aprimorar nosso modelo de previsão de linguagem natural. Os resultados mais relevantes e suas análises detalhadas estão disponíveis na seção 9 da documentação do projeto, "Análise dos Modelos". Para acessar essa seção, clique [aqui](#).

Para rodar o notebook a seguir corretamente, é necessário seguir alguns passos prévios. Primeiro, baixe os datasets disponíveis no link a seguir: [Datasets](https://drive.google.com/drive/folders/1bm6iQenyZ63gbw_s7j0md8UaHP84O0Vn?usp=sharing) (não é necessário todos, a princípio, somente os citados abaixo). Esses datasets não estão incluídos no notebook devido ao seu tamanho, por isso são ignorados pelo arquivo `.gitignore`. Todos eles devem ser colocados na pasta [`data`](data), localizada em `src/Notebook/Api_Sprint_4/data`.

Os arquivos necessários são:

- **processed_text_data.csv**: [Download](https://drive.google.com/file/d/1hCW3pliSKDWKyr0h0tvNBtUgtw9EsE3c/view?usp=drive_link) - dataset pré-pocessado.
- **extended_dataset.csv**: [Download](https://docs.google.com/spreadsheets/d/12q2Z3TthaaPSafgpridg4ikgAJiewhA_PuPuS6cMY60/edit?usp=drive_link) - dataset passado pelo pré-processamento e passado por um processo de equilíbrio de classes com dados sintéticos. 
- **cc.en.300.bin**: [Download](https://drive.google.com/file/d/1eM6TfyZIUt6YlCOus5C8nCU1UkmqRu2V/view?usp=sharing) - arquivo bin com palavras pré-treinadas para o FastText.
- **glove.6B.50d.txt**: [Download](https://drive.google.com/open?id=1xOZ1Gilf38-GtQ9u_ko8AB6XOBkt7UA7&usp=drive_copy) - contém palavras pré-treinadas em 50 dimensões.

Todos são necessários para rodar as células seguintes.

Outras recomendações:

- Mantenha o C++ do seu computador atualizado, pois determinadas bibliotecas são compiladas com essa linguagem.
- Caso queira conferir as versões mais atualizaas das bibliotecas, elas estão detalhadas no aquivo [``requirements.txt``](../../requirements.txt). Para usar este arquivo, basta rodar no terminal o comando: ``pip install -r requirements.txt``. Porém, a princípio, é possível ter acesso a todas as bilioteca ao rodar a célula de "Instalação e Importação das Bibliotecas" deste Notebook.

# Instalação e Importação das Bibliotecas

Na seção "Instalação e Importação das Bibliotecas", prepara-se o ambiente de codificação configurando e carregando as bibliotecas necessárias para o projeto. Esta seção é importante pelos seguintes motivos:

1. **Instalação de Bibliotecas**: Utilizam-se comandos como `%pip install nome_da_biblioteca` para instalar bibliotecas Python que não estão presentes por padrão no ambiente de execução, mas que são essenciais para a execução do código. Este comando mágico oferece uma integração mais direta com o IPython, garantindo que as instalações ocorram no kernel correto do Python utilizado pelo Jupyter.

2. **Importação de Bibliotecas**: Após a instalação, as bibliotecas são importadas usando o comando `import`, permitindo o acesso às funções e ferramentas disponibilizadas por elas. Comandos típicos incluem `import numpy as np` e `import pandas as pd`. Esta prática garante que todas as funcionalidades necessárias estejam disponíveis e prontas para uso nas seções subsequentes do notebook.

Esta seção é posicionada no início do notebook para assegurar que todas as dependências estejam corretamente configuradas antes de prosseguir com a análise de dados ou modelagem.

In [12]:
# Instalação de Bibliotecas Necessárias
%pip install pandas
%pip install numpy
%pip install nltk
%pip install emot
%pip install matplotlib
%pip install scikit-learn
%pip install gensim
%pip install seaborn
%pip install xgboost
%pip install scipy==1.11
%pip install fasttext
%pip install fasttext-wheel

!python -m spacy download en_core_web_md

%pip install --upgrade scikit-learn
%pip install --upgrade imbalanced-learn

In [ ]:
# Importação de Bibliotecas
import fasttext
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    roc_curve, 
    roc_auc_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report, 
    cohen_kappa_score
)
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from collections import Counter
import nltk


In [ ]:
# Baixar recursos necessários do nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Extração de Features

Neste notebook, exploramos diversos testes para determinar a melhor abordagem de vetorização no contexto do modelo de previsão de linguagem natural escolhido na sprint 3, que é o XGBoost. A decisão sobre o modelo foi detalhadamente analisada na seção 7.2 da documentação do projeto, intitulada "Resultados".

Aqui, focamos na comparação entre diferentes técnicas de vetorização. No decorrer deste notebook, você encontrará testes e avaliações que ajudaram a identificar a vetorização mais eficaz. Na seção "Extração de Features", você poderá observar o processo de pré-treinamento das seguintes técnicas de vetorização:

1. **TF-IDF (Term Frequency-Inverse Document Frequency)**
2. **GloVe com 50 dimensões (Global Vectors for Word Representation)**
3. **FastText**
4. **Part-of-Speech Tagging**
5. **N-grams**

## TD-IDF

O TF-IDF é uma técnica amplamente utilizada para transformar texto em uma representação numérica que pode ser alimentada a um algoritmo de aprendizado de máquina. Esta técnica calcula a frequência de uma palavra em um documento (Term Frequency - TF) e a pondera com a raridade dessa palavra em todos os documentos (Inverse Document Frequency - IDF). O resultado é uma métrica que reflete o quão importante uma palavra é para um documento específico, em relação a todo o corpus. No nosso projeto, o TF-IDF ajudou a capturar a relevância das palavras em contextos específicos, proporcionando uma base sólida para a modelagem.

In [ ]:
def compute_tfidf(dataframe_path, text_column):
    """
    Calcula a matriz TF-IDF para um DataFrame e coluna de texto fornecidos.

    Input:
    dataframe (pd.DataFrame): O DataFrame de entrada contendo os dados de texto.
    text_column (str): O nome da coluna contendo o texto processado.

    Output:
    pd.DataFrame: Um DataFrame contendo a matriz TF-IDF.
    """

    dataframe = pd.read_csv(dataframe_path)

    # Cria um Vetorizador TF-IDF
    tfidf_vectorizer = TfidfVectorizer()

    # Ajusta e transforma o texto processado
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe[text_column])

    # Converte a matriz TF-IDF para um DataFrame para melhor legibilidade
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    
    return tfidf_df


In [ ]:
# Load the uploaded CSV file
file_path = './data/processed_text_data.csv'

# Calcula a matriz TF-IDF
tfidf_df = compute_tfidf(file_path, 'processed_text')

tfidf_df.head()

## GloVe

GloVe é uma técnica de vetorização de palavras que cria vetores densos a partir de grandes corpora de texto. Ele é treinado para capturar a co-ocorrência de palavras em um espaço vetorial, onde palavras semanticamente similares estão próximas. Usamos a versão com 50 dimensões para representar cada palavra em nosso corpus, proporcionando uma representação semântica que vai além das frequências de palavras simples. Essa técnica é especialmente útil para capturar contextos mais sutis e relações semânticas entre palavras que o TF-IDF pode não captar.

In [ ]:
def load_glove_model(file_path):
    '''
    Carrega o modelo GloVe de um arquivo de texto.

    Input:
    - file_path (str): Caminho para o arquivo de vetores GloVe.

    Output:
    - dict: Dicionário onde as chaves são palavras e os valores são vetores.
    '''
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=float)
            glove_model[word] = vector
    return glove_model

In [ ]:
def process_text_to_vectors_glove(csv_input_path, text_column, glove_file_path):
    '''
    Processa um arquivo CSV, transforma o texto em vetores de palavras usando GloVe, e salva o resultado em um novo arquivo CSV.

    Input:
    - csv_input_path (str): Caminho para o arquivo CSV de entrada.
    - text_column (str): Nome da coluna que contém o texto processado.
    - glove_file_path (str): Caminho para o arquivo de vetores GloVe.

    Output:
    - pd.DataFrame: DataFrame contendo os textos e seus vetores correspondentes divididos em colunas.
    '''
    # Carregar o modelo GloVe
    glove_model = load_glove_model(glove_file_path)

    # Função para obter o vetor de uma palavra
    def get_word_vector(word):
        return glove_model.get(word, np.zeros(50))

    # Função para calcular a média dos vetores das palavras em um texto
    def average_word_vectors(text):
        words = text.split()
        if len(words) == 0:
            return np.zeros(50)
        vectors = np.array([get_word_vector(word) for word in words])
        return np.mean(vectors, axis=0)

    # Carregar os dados do arquivo CSV
    data = pd.read_csv(csv_input_path)

    # Calcular o vetor médio para cada linha de texto
    data['vectors'] = data[text_column].apply(average_word_vectors)

    # Dividir os vetores em colunas separadas
    vector_columns = [f'vector_{i}' for i in range(50)]
    vectors_df = pd.DataFrame(data['vectors'].tolist(), columns=vector_columns)

    # Concatenar os vetores de volta ao DataFrame original
    data = pd.concat([data, vectors_df], axis=1)

    # Manter apenas as colunas de sentimento e vetores
    result_data = data[['sentiment'] + vector_columns]

    return result_data

## FastText


O FastText, desenvolvido pelo Facebook, é uma ferramenta que também gera vetores de palavras, mas com a vantagem adicional de considerar subpalavras e morfologia. Utilizamos vetores pré-treinados do FastText, que oferecem uma representação mais robusta para palavras raras e variações morfológicas, como sufixos e prefixos. Isso melhora a capacidade do modelo de lidar com palavras novas ou raras, capturando nuances que outros métodos podem não conseguir. Esta característica torna o FastText particularmente eficaz em nosso modelo para tratar a diversidade linguística presente nos dados.

In [ ]:
def text_to_vector(text, model):
    """
    Converte um texto em um vetor usando a média dos embeddings de palavras.

    Args:
    text (str): Texto de entrada.
    model: Modelo fastText carregado.

    Returns:
    np.array: Vetor que representa o texto.
    """
    words = text.split()
    word_vectors = [model.get_word_vector(word) for word in words if word in model.words]
    if len(word_vectors) == 0:
        return np.zeros(model.get_dimension())
    return np.mean(word_vectors, axis=0)

def vectorize_text_with_fasttext(dataframe, text_column, fasttext_model_path):
    """
    Converte uma coluna de texto em embeddings usando um modelo fastText.

    Args:
    dataframe (pd.DataFrame): DataFrame contendo a coluna de texto.
    text_column (str): Nome da coluna de texto a ser convertida.
    fasttext_model_path (str): Caminho para o modelo fastText pré-treinado.

    Returns:
    np.array: Matriz de embeddings onde cada linha corresponde a um texto.
    """
    # Carregar o modelo fastText
    if not os.path.exists(fasttext_model_path):
        raise FileNotFoundError(f"Modelo fastText não encontrado no caminho: {fasttext_model_path}")
    fasttext_model = fasttext.load_model(fasttext_model_path)

    # Verificar se a coluna de texto existe e está no formato correto
    if text_column not in dataframe.columns:
        raise KeyError(f"A coluna '{text_column}' não foi encontrada no DataFrame.")
    if not pd.api.types.is_string_dtype(dataframe[text_column]):
        raise ValueError(f"A coluna '{text_column}' deve ser do tipo string.")
    
    # Converter cada texto no dataset para um vetor de embeddings
    dataframe['embedding'] = dataframe[text_column].apply(lambda x: text_to_vector(x, fasttext_model))

    # Verificar se todos os textos foram convertidos corretamente
    if dataframe['embedding'].apply(lambda x: len(x) != fasttext_model.get_dimension()).any():
        raise ValueError("Alguns textos não foram convertidos corretamente em embeddings.")

    # Extrair as embeddings como matriz de features
    X = np.vstack(dataframe['embedding'].values)

    return X

## Part-of-Speech Tagging

O Part-of-Speech Tagging (POS Tagging) é uma técnica que atribui rótulos gramaticais a cada palavra em uma frase, indicando sua função, como substantivo, verbo, adjetivo, etc. Treinamos nosso próprio modelo de POS Tagging usando um banco de dados específico para capturar as nuances sintáticas de nosso corpus. Essa técnica é valiosa para entender a estrutura gramatical e pode fornecer insights sobre como diferentes partes do discurso influenciam o significado e o contexto em que as palavras são usadas. A inclusão de informações gramaticais enriquece a representação textual, auxiliando o modelo a diferenciar entre palavras com múltiplos significados dependendo de seu uso gramatical.

In [ ]:
def pos_tagging(text):
    """
    Realiza o POS tagging em um texto e retorna as tags.

    Args:
    text (str): Texto de entrada.

    Returns:
    list: Lista de tags POS.
    """
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return [tag for word, tag in pos_tags]

In [ ]:
def pos_features(dataframe, text_column):
    """
    Converte a coluna de texto em features baseadas nas tags de POS.

    Args:
    dataframe (pd.DataFrame): DataFrame contendo a coluna de texto.
    text_column (str): Nome da coluna de texto a ser convertida.

    Returns:
    pd.DataFrame: DataFrame com as tags de POS convertidas em features.
    """
    # Realizar POS tagging para cada texto
    dataframe['pos_tags'] = dataframe[text_column].apply(pos_tagging)
    
    # Criar uma lista de todas as tags de POS possíveis
    all_tags = [tag for tags in dataframe['pos_tags'] for tag in tags]
    unique_tags = list(set(all_tags))

    # Converter tags de POS em features de contagem
    def pos_tag_counts(tags):
        tag_count = Counter(tags)
        return [tag_count.get(tag, 0) for tag in unique_tags]

    pos_features = dataframe['pos_tags'].apply(pos_tag_counts)
    pos_features_df = pd.DataFrame(pos_features.tolist(), columns=[f'pos_{tag}' for tag in unique_tags])

    return pos_features_df

## N-grams

Os N-grams são sequências contíguas de n itens de um dado texto, onde "n" pode variar. Neste projeto, utilizamos um banco de dados próprio para treinar nossos N-grams, capturando padrões de co-ocorrência entre palavras em nosso corpus. Eles são especialmente úteis para capturar a ordem e a proximidade de palavras, oferecendo uma visão mais profunda sobre frases e expressões comuns que podem não ser evidentes em métodos de vetorização baseados em palavras únicas. Por exemplo, a frase “análise de sentimentos” como um bigrama (n=2) pode capturar um contexto mais específico do que as palavras “análise” e “sentimentos” individualmente.

In [ ]:
def generate_ngrams(dataframe, text_column, ngram_range=(1,2)):
    """
    Converte uma coluna de texto em features de N-grams usando TF-IDF.

    Args:
    dataframe (pd.DataFrame): DataFrame contendo a coluna de texto.
    text_column (str): Nome da coluna de texto a ser convertida.
    ngram_range (tuple): O intervalo de N para N-grams (exemplo: (1, 2) para unigrams e bigrams).

    Returns:
    pd.DataFrame: DataFrame com a representação TF-IDF dos N-grams.
    """
    # Cria um Vetorizador TF-IDF para N-grams
    tfidf_vectorizer = TfidfVectorizer(ngram_range=ngram_range)

    # Ajusta e transforma o texto processado
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe[text_column])

    # Converte a matriz TF-IDF para um DataFrame para melhor legibilidade
    ngram_features_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

    return ngram_features_df

# Criação dos Datasets para treinamento dos modelos

Nesta seção, nos concentramos em criar diversos datasets derivados do "processed_text_data.csv" com o objetivo de treinar nossos modelos utilizando diferentes bancos de dados e técnicas de vetorização. A diversidade de datasets permite uma avaliação mais abrangente e robusta dos modelos de aprendizado de máquina. Para isso, exploramos quatro principais conjuntos de dados, cada um com características únicas que ajudam a testar a eficácia do modelo sob diferentes condições:

1. **processed_text_data.csv**
   Este é o dataset base que contém os textos processados com suas respectivas classificações de sentimento. Ele serve como ponto de partida para todos os experimentos subsequentes. O dataset é usado diretamente para treinar os modelos sem qualquer modificação adicional, proporcionando uma linha de base para comparar os efeitos de outras abordagens de balanceamento.

2. **extended_dataset.csv**
   Para lidar com o desbalanceamento do dataset original, criamos o "extended_dataset.csv". Este dataset foi ampliado utilizando técnicas de Inteligência Artificial para gerar novos dados sintéticos baseados nos dados originais. Com isso, conseguimos igualar a quantidade de exemplos negativos, neutros e positivos, proporcionando um dataset balanceado que permite ao modelo aprender de forma mais equilibrada e eficaz sobre cada classe de sentimento.

3. **balanced_dataset_with_undersampling**
   Função que cria dataset (normalmente, uma variante do "extended_dataset.csv"), onde aplicamos undersampling para balancear o número de exemplos em cada classe. Especificamente, removemos uma quantidade igual de exemplos positivos e neutros para que o número de exemplos seja equivalente ao de exemplos negativos. Esta abordagem é útil para evitar que o modelo aprenda vieses de classes majoritárias, mantendo o foco em dados representativos e relevantes.

4. **balanced_dataset_with_oversampling**
   Função que cria dataset (normalmente, uma variante do "extended_dataset.csv") onde aplica-se oversampling para aumentar o número de exemplos sintéticos em classes com menos exemplos (como para os testes una-se neutros e positivos, seria a classe negativa). Ao contrário do undersampling, que remove exemplos, o oversampling cria novos exemplos para as classes minoritárias. Isso ajuda a garantir que o modelo tenha uma exposição equilibrada a todas as classes durante o treinamento, promovendo uma melhor generalização e precisão.

## Abordagem de Diversidade de Testes

Ao criar esses diferentes datasets, buscamos maximizar a diversidade de testes para o nosso modelo de previsão de linguagem natural. Cada dataset oferece uma perspectiva única sobre como os modelos podem se comportar com diferentes distribuições de dados e técnicas de vetorização. Essa estratégia permite uma avaliação mais completa e rigorosa, ajudando a identificar a melhor abordagem para nosso problema de classificação de sentimentos.


A Função a seguir carrega dois arquivos CSV, adiciona a coluna 'sentiment' do primeiro arquivo ao segundo arquivo.

In [ ]:
def merge_sentiment_column(vectorizer, dataframe):
    """
    Carrega dois arquivos CSV, adiciona a coluna 'sentiment' do primeiro arquivo
    ao segundo arquivo, e salva o resultado em um novo arquivo CSV.

    Args:
    dataframe (str): Caminho para o arquivo CSV contendo a coluna 'sentiment'.
    vectorizer (str): Caminho para o arquivo CSV contendo os vetores de palavras.
    
    Returns:
    vectorizer: Arquivo CSV contendo a coluna 'sentiment' e o contendo os vetores de palavras.
    """
    
    # Carregar os arquivos CSV
    classification_text_data = pd.read_csv(dataframe)

    # Selecionar a coluna 'sentiment' do arquivo classification_text_data
    sentiment_column = classification_text_data['sentiment']

    if 'sentiment' in vectorizer.columns:
        vectorizer = vectorizer.drop('sentiment', axis=1)

    # Adicionar a coluna 'sentiment' no arquivo vectorizer na primeira coluna (coluna 0)
    vectorizer.insert(0, 'sentiment', sentiment_column)
    
    return vectorizer

O código a seguir cria um banco de dados que remove dados positivos e neutros para equilibrar com os negativos

In [ ]:
def balanced_dataset_with_undersampled(file_path):
    data_balanced = pd.read_csv(file_path)

    # Filtrando os dados positivos e neutros
    data_positive = data_balanced[data_balanced['sentiment'] == 1]
    data_neutral = data_balanced[data_balanced['sentiment'] == 0]

    # Selecionando os índices das últimas 665 linhas de cada subset
    indices_to_drop = data_positive.tail(665).index.tolist() + \
                    data_neutral.tail(665).index.tolist()

    # Removendo as linhas do DataFrame original
    data_balanced = data_balanced.drop(indices_to_drop)
    return data_balanced

In [ ]:
data_balanced = balanced_dataset_with_undersampled('./data/extended_dataset.csv')
data_balanced.to_csv('./data/extended_dataset_undersampled.csv', index=False)

O código a seguir pretende equilibrar o dataset ao aumentar sinteticamente os dados negativos

In [ ]:
def balance_dataset_with_oversampled(file_path):
    """
    Balanceia o dataset para que a quantidade de exemplos da classe negativa seja igual à soma das quantidades das classes positiva e neutra.

    Inputs:
        file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.

    Outputs:
        pd.DataFrame: DataFrame balanceado com quantidades iguais de classes negativas e a soma de positivas e neutras.
    """
    
        # Carregar os dados
    data = pd.read_csv(file_path)

    # Verificar a presença da coluna 'sentiment'
    if 'sentiment' not in data.columns:
        raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

    # Separar as classes
    data_negative = data[data['sentiment'] == -1]
    data_positive_neutral = data[data['sentiment'] != -1]

    # Contar a quantidade de exemplos nas classes positivas e neutras combinadas
    num_positive_neutral = len(data_positive_neutral)

    # Fazer o oversampling da classe negativa para igualar a quantidade das classes positivas e neutras combinadas
    data_negative_oversampled = resample(data_negative,
                                         replace=True,
                                         n_samples=num_positive_neutral,
                                         random_state=42)

    # Combinar as classes balanceadas
    data_balanced = pd.concat([data_negative_oversampled, data_positive_neutral])

    return data_balanced


In [ ]:
data_balanced = balance_dataset_with_oversampled('./data/extended_dataset.csv')
data_balanced.to_csv('./data/extended_dataset_oversampled.csv', index=False)

# Treinamento dos modelos

Nesta seção, apresentamos as funções utilizadas para treinar nossos modelos de previsão de sentimentos utilizando o algoritmo XGBoost. Desenvolvemos duas abordagens específicas que dividem o problema de classificação em tarefas binárias. Essa estratégia permite que o modelo se concentre em distinguir entre classes específicas em cada etapa. Abaixo, descrevemos essas duas funções em detalhes:

### negativo vs não negativo
**Descrição:**  
Nesta função, o objetivo é prever se uma sentença é negativa ou não. Para isso, os exemplos de sentimentos positivos e neutros são combinados em uma única categoria chamada "não-negativos". Isso simplifica o problema em uma tarefa binária, onde o modelo aprende a diferenciar entre sentenças negativas e todas as outras.

In [ ]:
def train_xgboost_model_negative_vs_rest(data_for_training_path):
    """
    Treina um modelo XGBoost para classificação binária (negativa ou não negativa).

    Inputs:
        data_for_training_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados de treinamento a partir de um arquivo CSV
        data_for_training = pd.read_csv(data_for_training_path)

        if 'id' in data_for_training.columns:
            data_for_training = data_for_training.drop(columns=['id'])

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data_for_training.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Dividir os dados em features (X) e target (y)
        X = data_for_training.drop('sentiment', axis=1)
        y = data_for_training['sentiment']

        # Reclassificar as classes para binário
        y = y.apply(lambda x: 1 if x == -1 else 0)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Aplicar SMOTE para balancear as classes no conjunto de treinamento
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        # Configurar o modelo XGBoost
        model = xgb.XGBClassifier(max_depth=6, eta=0.3, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

        # Realizar validação cruzada
        cv_scores = cross_val_score(model, X_train_res, y_train_res, cv=5, scoring='f1_weighted')

        # Treinar o modelo
        model.fit(X_train_res, y_train_res)

        # Fazer previsões
        y_pred = model.predict(X_test)

        # Verificar os rótulos únicos nas previsões
        print("Rótulos únicos nas previsões:", np.unique(y_pred))

        # Calcular e exibir o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted'),
            'kappa': cohen_kappa_score(y_test, y_pred),
            'mean_cross_val_f1': cv_scores.mean(),
            'std_cross_val_f1': cv_scores.std()
        }

        print(metrics)
        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

### positivo vs neutro 
**Descrição:**  
Após determinar que uma sentença não é negativa usando o primeiro modelo, esta função é aplicada para distinguir entre sentenças positivas e neutras. Neste caso, os exemplos de sentimentos negativos são excluídos do dataset, focando o treinamento apenas na diferenciação entre as classes positivas e neutras.

In [ ]:
def train_xgboost_model_positive_vs_neutral(file_path):
    """
    Treina um modelo XGBoost para classificação binária (neutra ou positiva), excluindo frases negativas.

    Inputs:
        file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados
        data = pd.read_csv(file_path)

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Filtrar para remover frases negativas
        data = data[data['sentiment'] != -1]

        # Dividir os dados em features (X) e target (y)
        X = data.drop('sentiment', axis=1)
        y = data['sentiment']

        # Reclassificar as classes para binário
        y = y.apply(lambda x: 1 if x == 1 else 0)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Aplicar SMOTE para balancear as classes no conjunto de treinamento
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        # Configurar o modelo XGBoost
        model = xgb.XGBClassifier(max_depth=6, eta=0.3, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

        # Realizar validação cruzada
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_res, y_train_res, cv=cv, scoring='f1_weighted')

        # Treinar o modelo
        model.fit(X_train_res, y_train_res)

        # Fazer previsões
        y_pred = model.predict(X_test)

        # Verificar os rótulos únicos nas previsões
        print("Rótulos únicos nas previsões:", np.unique(y_pred))

        # Calcular e exibir o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted'),
            'kappa': cohen_kappa_score(y_test, y_pred),
            'mean_cross_val_f1': cv_scores.mean(),
            'std_cross_val_f1': cv_scores.std()
        }

        print(metrics)
        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

# Pipeline

Nesta seção, detalhamos o pipeline completo utilizado para testar diferentes técnicas de vetorização em combinação com os variados datasets. O objetivo é identificar a configuração mais eficaz para a previsão de sentimentos usando o modelo XGBoost. Através desta abordagem, buscamos explorar a combinação ideal de vetorização e dataset para maximizar a performance do modelo.

## TD_IDF

### TD-IDF Pipeline

In [ ]:
def pipeline_complete_tdidf(dataframe, text_column):
    """
    Pipeline para vetorização via TD-IDF e treino do modelo XGBoost.

    Input:
    dataframe (pd.DataFrame): O DataFrame de entrada contendo os dados de texto.
    text_column (str): O nome da coluna contendo o texto processado.
    """
    
    # Vetorização via TD-IDF
    tfidf_df = compute_tfidf(dataframe, text_column)

    # Função que cria o banco de dados de treino:
    data_for_training = merge_sentiment_column(tfidf_df, dataframe)

    # Salvando o DataFrame resultante para treino
    data_for_training_path = './data/teste1_td-idf_data.csv'
    data_for_training.to_csv(data_for_training_path, index=False)

    print("Métricas para o Modelo Negativo vs Não Negativo")
    # Função que treina os dados com o modelo TD-IDF e retorna as métricas
    metrics_neg_vs_rest = train_xgboost_model_negative_vs_rest(data_for_training_path)

    print("Métricas para o Modelo Neutro vs Positivo")
    # Função que treina os dados com o modelo TD-IDF e retorna as métricas
    metrics_pos_vs_neutral = train_xgboost_model_positive_vs_neutral(data_for_training_path)

    return metrics_neg_vs_rest, metrics_pos_vs_neutral

### TD-IDF Testes

In [ ]:
# Teste feito com os dados pré-processados

dataframe = "./data/processed_text_data.csv"
text_column = "processed_text"
# model_save_path = "./models/xgboost_model_td-idf.pkl"

pipeline_complete_tdidf(dataframe, text_column)

In [ ]:
# Teste feito com os dados balanceados (com quantidades equivalentes de negativos, positivos e neutros)

dataframe = "./data/extended_dataset.csv"
text_column = "processed_text"

pipeline_complete_tdidf(dataframe, text_column)

In [ ]:
# Teste feito com os dados balanceados (com quantidades equivalentes de negativos, positivos e neutros) e então equilibrados (diminuindo os positivos/neutros)
dataframe = "./data/extended_dataset_oversampled.csv"
text_column = "processed_text"
pipeline_complete_tdidf(dataframe, text_column)

In [ ]:
# Teste feito com os dados balanceados e então equilibrados (aumentando os negativos)
dataframe = "./data/extended_dataset_undersampled.csv"
pipeline_complete_tdidf(dataframe, text_column)

## GloVe 50 dimensões

### Pipeline GloVe

In [ ]:
def pipeline_complete_glove_undersampling(csv_input_path, text_column, glove_file_path):
    
    vector_data = process_text_to_vectors_glove(csv_input_path, text_column, glove_file_path)
    
    vector_data.to_csv('./data/glove_data.csv', index=False)

    vector_data = balanced_dataset_with_undersampled('./data/glove_data.csv')

    vector_data.to_csv('./data/glove_data.csv', index=False)

    print("Métricas para o Modelo Negativo vs Não Negativo")
    train_xgboost_model_negative_vs_rest('./data/glove_data.csv')
    
    print("Métricas para o Modelo Neutro vs Positivo")
    train_xgboost_model_positive_vs_neutral('./data/glove_data.csv')

### Testes GloVe

In [ ]:
csv_input_path = './data/extended_dataset.csv'
text_column = 'processed_text'
glove_file_path = './data/glove.6B.50d.txt'

pipeline_complete_glove_undersampling(csv_input_path, text_column, glove_file_path)

## FastText

In [ ]:
def train_xgboost_model_with_fasttext(data_for_training_path, fasttext_model_path):
    """
    Treina um modelo XGBoost para classificação binária usando embeddings de fastText.

    Inputs:
        data_for_training_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
        fasttext_model_path (str): Caminho para o modelo pré-treinado fastText.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados de treinamento a partir de um arquivo CSV
        data_for_training = pd.read_csv(data_for_training_path)

        if 'id' in data_for_training.columns:
            data_for_training = data_for_training.drop(columns=['id'])

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data_for_training.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Realizar a vetorização com fastText
        X = vectorize_text_with_fasttext(data_for_training, 'processed_text', fasttext_model_path)

        # Remover colunas não numéricas como 'comment' e qualquer outra coluna de texto
        non_numeric_columns = data_for_training.select_dtypes(include=[object]).columns
        data_for_training.drop(columns=non_numeric_columns, inplace=True)
        
        # Adicionar os embeddings ao DataFrame original
        embeddings_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
        data_for_training = pd.concat([data_for_training.reset_index(drop=True), embeddings_df], axis=1)

        # Verificar se só restam colunas numéricas no DataFrame
        non_numeric_columns = data_for_training.drop(columns=['sentiment']).select_dtypes(exclude=[np.number]).columns
        if len(non_numeric_columns) > 0:
            raise ValueError(f"O DataFrame contém colunas não numéricas: {list(non_numeric_columns)}")

        # Salvar o novo DataFrame vetorizado para ser usado na função original
        processed_data_path = data_for_training_path.replace('.csv', '_processed.csv')
        data_for_training.to_csv(processed_data_path, index=False)

        print("Métricas para o Modelo Negativo vs Não Negativo")
        # Chamar a função original do XGBoost com os dados vetorizados
        metrics = train_xgboost_model_negative_vs_rest(processed_data_path)

        print("Métricas para o Modelo Neutro vs Positivo")
        metrics = train_xgboost_model_positive_vs_neutral(processed_data_path)
        
        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

In [ ]:
# Exemplo de uso:
data_for_training_path = './data/processed_text_data.csv'
fasttext_model_path = './data/cc.en.300.bin'

# Treinar e avaliar o modelo com a integração do fastText
train_xgboost_model_with_fasttext(data_for_training_path, fasttext_model_path)


In [ ]:
# Exemplo de uso:
data_for_training_path = './data/extended_dataset_undersampled.csv'
fasttext_model_path = './data/cc.en.300.bin'

# Treinar e avaliar o modelo com a integração do fastText
train_xgboost_model_with_fasttext(data_for_training_path, fasttext_model_path)

## Part-of-Speech Tagging

In [ ]:
def train_xgboost_model_with_pos(data_for_training_path):
    """
    Treina um modelo XGBoost para classificação binária usando features de POS tagging.

    Inputs:
        data_for_training_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados de treinamento a partir de um arquivo CSV
        data_for_training = pd.read_csv(data_for_training_path)

        if 'id' in data_for_training.columns:
            data_for_training = data_for_training.drop(columns=['id'])

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data_for_training.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Realizar a conversão de POS tags em features
        pos_features_df = pos_features(data_for_training, 'processed_text')

        # Remover colunas não numéricas como 'comment' e qualquer outra coluna de texto
        non_numeric_columns = data_for_training.select_dtypes(include=[object]).columns
        data_for_training.drop(columns=non_numeric_columns, inplace=True)

        # Adicionar as features de POS ao DataFrame original
        data_for_training = pd.concat([data_for_training.reset_index(drop=True), pos_features_df.reset_index(drop=True)], axis=1)

        # Verificar se só restam colunas numéricas no DataFrame
        non_numeric_columns = data_for_training.drop(columns=['sentiment']).select_dtypes(exclude=[np.number]).columns
        if len(non_numeric_columns) > 0:
            raise ValueError(f"O DataFrame contém colunas não numéricas: {list(non_numeric_columns)}")

        # Salvar o novo DataFrame com as features de POS para ser usado na função original
        processed_data_path = data_for_training_path.replace('.csv', '_processed_with_pos.csv')
        data_for_training.to_csv(processed_data_path, index=False)

        print("Métricas para o Modelo Negativo vs Não Negativo")
        # Chamar a função original do XGBoost com os dados vetorizados
        metrics = train_xgboost_model_negative_vs_rest(processed_data_path)

        print("Métricas para o Modelo Neutro vs Positivo")
        metrics = train_xgboost_model_positive_vs_neutral(processed_data_path)
        
        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None


In [ ]:
# Exemplo de uso:
data_for_training_path = './data/processed_text_data.csv'

# Treinar e avaliar o modelo com a integração do POS Tagging
train_xgboost_model_with_pos(data_for_training_path)

In [ ]:
data_for_training_path = './data/extended_dataset_undersampled.csv'

# Treinar e avaliar o modelo com a integração do POS Tagging
train_xgboost_model_with_pos(data_for_training_path)

## N-grams

In [ ]:
def train_xgboost_model_with_ngrams(data_for_training_path, ngram_range=(1,2)):
    """
    Treina um modelo XGBoost para classificação binária usando features de N-grams.

    Inputs:
        data_for_training_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
        ngram_range (tuple): O intervalo de N para N-grams (exemplo: (1, 2) para unigrams e bigrams).

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados de treinamento a partir de um arquivo CSV
        data_for_training = pd.read_csv(data_for_training_path)

        if 'id' in data_for_training.columns:
            data_for_training = data_for_training.drop(columns=['id'])

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data_for_training.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Gerar as features de N-grams
        ngram_features_df = generate_ngrams(data_for_training, 'processed_text', ngram_range)

        # Remover colunas não numéricas como 'comment' e qualquer outra coluna de texto
        non_numeric_columns = data_for_training.select_dtypes(include=[object]).columns
        data_for_training.drop(columns=non_numeric_columns, inplace=True)

        # Adicionar as features de N-grams ao DataFrame original
        data_for_training = pd.concat([data_for_training.reset_index(drop=True), ngram_features_df.reset_index(drop=True)], axis=1)

        # Verificar se só restam colunas numéricas no DataFrame
        non_numeric_columns = data_for_training.drop(columns=['sentiment']).select_dtypes(exclude=[np.number]).columns
        if len(non_numeric_columns) > 0:
            raise ValueError(f"O DataFrame contém colunas não numéricas: {list(non_numeric_columns)}")

        # Salvar o novo DataFrame com as features de N-grams para ser usado na função original
        processed_data_path = data_for_training_path.replace('.csv', '_processed_with_ngrams.csv')
        data_for_training.to_csv(processed_data_path, index=False)

        print("Métricas para o Modelo Negativo vs Não Negativo")
        # Chamar a função original do XGBoost com os dados vetorizados
        metrics = train_xgboost_model_negative_vs_rest(processed_data_path)

        print("Métricas para o Modelo Neutro vs Positivo")
        metrics = train_xgboost_model_positive_vs_neutral(processed_data_path)

        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None


In [ ]:
# Exemplo de uso:
data_for_training_path = './data/processed_text_data.csv'

# Treinar e avaliar o modelo com a integração de N-grams
metrics = train_xgboost_model_with_ngrams(data_for_training_path, ngram_range=(1,2))

In [ ]:
# Exemplo de uso:
data_for_training_path = './data/extended_dataset_undersampled.csv'

# Treinar e avaliar o modelo com a integração de N-grams
metrics = train_xgboost_model_with_ngrams(data_for_training_path, ngram_range=(1,2))